# Age, Gender and Emotion Detection

### Let's load our classfiers

In [1]:
from pathlib import Path
import cv2
import dlib
import sys
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
from keras.models import load_model
from keras.preprocessing.image import img_to_array


classifier = load_model('./model/emotion_little_vgg_2.h5')
pretrained_model = './Weights/weights.28-3.73.hdf5'

### Detecting Age, Gender, Emotion of people in any Image

In [2]:
from os import listdir
from os.path import isfile, join
import os
import cv2


image_path = "./images/"

emotion_classes = {0: 'Angry ' , 1: 'Fear', 2: 'Happy' , 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}


def draw_label(image, point, label, font=cv2.FONT_HERSHEY_PLAIN,
               font_scale=1, thickness=1):
    
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
   
    x, y = point
    
    cv2.rectangle(image, (x+size[0], y), (x, y-size[1]), (0, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (100, 50, 255), thickness, lineType=cv2.LINE_AA)#the information text
    

# Define our model parameters
depth = 16
k = 8      
weight_file = None
margin = 0.4    #for imdb
image_dir = None

if not weight_file:
    weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                              cache_dir=Path(sys.argv[0]).resolve().parent)

img_size = 64 

model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)

detector = dlib.get_frontal_face_detector() 

image_names = [f for f in listdir(image_path) if isfile(join(image_path, f))]


for image_name in image_names:
    frame = cv2.imread("./images/" + image_name) 
    
    preprocessed_faces_emo = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    img_h, img_w, _ = np.shape(input_img)  
    
    detected = detector(frame, 1) 

    faces = np.empty((len(detected), img_size, img_size, 3))
    
    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):  
           
            x1 = d.left()     
            y1 = d.top()       
            x2 = d.right() + 1  
            y2 = d.bottom() + 1 
            w = d.width()      
            h = d.height()    

            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
           
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            
            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) # change to greyscale
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
            face_gray_emo = face_gray_emo.astype("float") / 255.0    
            face_gray_emo = img_to_array(face_gray_emo)
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)

       
       # make a prediction for Age and Gender
        results = model.predict(np.array(faces))
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1) 
        predicted_ages = results[1].dot(ages).flatten() 

        emo_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])
            
    
      # draw results
        for i, d in enumerate(detected):
            label = "{}, {}, {}".format(int(predicted_ages[i]),
                                "Female" if predicted_genders[i][0] > 0.5 else "Male", emo_labels[i] )
           
        draw_label(frame, (d.left(), d.top()), label)

    cv2.imshow("A,G,E Detector", frame) 
    filename = "output_images/"+image_name
    
    cv2.imwrite(filename,frame) 
    
    
    #we need to pause execution, as the window will be destroyed when the script stops, 
    #so we use cv2.waitKey to hold the window until a key is pressed, and after that,
    #we destroy the window and exit the script.
    
    cv2.waitKey(0)  
    cv2.destroyAllWindows()   

### Detection with webcam

In [3]:
from os import listdir
from os.path import isfile, join
import os
import cv2


emotion_classes = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}


def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (0, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (100, 50, 255), thickness, lineType=cv2.LINE_AA)
    

# Define our model parameters
depth = 16
k = 8
weight_file = None
margin = 0.4
image_dir = None
 
if not weight_file:
    weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                                cache_dir=Path(sys.argv[0]).resolve().parent)
img_size = 64
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)

detector = dlib.get_frontal_face_detector()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  
  

    preprocessed_faces_emo = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)
    detected = detector(frame, 1)
  
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            
            faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
            face_gray_emo = face_gray_emo.astype("float") / 255.0
            face_gray_emo = img_to_array(face_gray_emo)
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)

        # make a prediction for Age and Gender
        results = model.predict(np.array(faces))
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()

        # make a prediction for Emotion 
        emo_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])
        
        # draw results
        for i, d in enumerate(detected):
            label = "{}, {}, {}".format(int(predicted_ages[i]),
                                        "Female" if predicted_genders[i][0] > 0.5 else "Male", emo_labels[i])
            draw_label(frame, (d.left(), d.top()), label)

    cv2.imshow("A,G,E Detector", frame)
    if cv2.waitKey(5) == 27:  # ESC key press
        break

cap.release() 
cv2.destroyAllWindows()      